# testing ground

In [5]:
%matplotlib inline

from pykat import finesse
from pykat import finesse
from pykat.commands import *
import matplotlib.pyplot as plt
import math

pykat.init_pykat_plotting(dpi=90)
finesse
kat = finesse.kat()
kat2 = finesse.kat()
kat.verbose = False
kat2.verbose = False

laserpower = "1.2M"
cavityspace = 0.6
phi = 0
Rc = 0.3
Refl = 0.997
Trans = 0.003
xbeta = 0.0  # pitch/yaw for mirror

code_gaussian = f"""
l laser {laserpower} 0 node0

# TODO: research about tem -> changes output!!
#tem laser 0 1 1 0
#maxtem 6
s prespace 0.5 node0 node1
m fixedmirror {Refl} {Trans} 0 node1 node2
s cavityspace {cavityspace} node2 node3
m piezomirror {Refl} {Trans} {phi} node3 node4
beam beamprofile node4
xaxis beamprofile x lin -2 2 100
x2axis beamprofile y lin -2 2 100

attr fixedmirror Rc {-Rc}
attr piezomirror Rc {Rc}
"""

code_planewave = f"""
l laser {laserpower} 0 node0
s prespace 0.5 node0 node1
m fixedmirror {Refl} {Trans} 0 node1 node2
s cavityspace {cavityspace} node2 node3
m piezomirror {Refl} {Trans} {phi} node3 node4
noxaxis
pd end node4
attr fixedmirror Rc {-Rc}
attr piezomirror Rc {Rc}

"""

kat.parse(code_gaussian)
kat2.parse(code_planewave)

# create a file
file = open("plot.csv", "w")

# Concatinate to this
output = f"phaseshift in deg, plane, beam\n"

for i in range(0, 361, 10):

    kat.piezomirror.phi = i
    kat2.piezomirror.phi = i
    out_gaus = kat.run()
    out_plane = kat2.run()

    steps_x = kat.xaxis.steps
    steps_y = kat.x2axis.steps

    beam_center = out_gaus["beamprofile"][steps_x // 2][steps_y // 2]
    if i % 180 == 0:
        print("===================================================")

    # print(f"at\t{kat2.piezomirror.phi}°\tshift:\t{out_plane.y[0][0]:.2e} Watts \t\t >>>>>>> \t\t {beam_center:.4e} AU")

    output += f"{i}, {out_plane.y[0][0]}, {beam_center}\n"

    # out2.plot()
    # i = 0
    ## making the plot
    # f = plt.figure(figsize=(6, 6))  # making sure the image is square
    # plt.grid(False)
    # plt.pcolormesh(out.x, out.y, out["beamprofile"], shading="auto")
    # plt.xlabel("x position [# spotsizes]")
    # plt.ylabel("y position [# spotsizes]")
    # f.savefig(f"foo{i}.jpg", bbox_inches='tight')

print(output)
file.write(output)
file.close()
""

phaseshift in deg, plane, beam
0, 1200000.0, 1714390.04694572
10, 89.8039921954767, 1714343.11024214
20, 23.1503125147703, 1714207.97574362
30, 10.8323997075252, 1714000.97899885
40, 6.55437431905477, 1713747.12816501
50, 4.61486501655445, 1713477.06832329
60, 3.6108216324324, 1713223.37264105
70, 3.06687350076824, 1713016.61360233
80, 2.79231672325961, 1712881.68813904
90, 2.70811826151847, 1712834.83398993
100, 2.79231672325961, 1712881.68813904
110, 3.06687350076824, 1713016.61360233
120, 3.6108216324324, 1713223.37264105
130, 4.61486501655445, 1713477.06832329
140, 6.55437431905476, 1713747.12816501
150, 10.8323997075252, 1714000.97899885
160, 23.1503125147703, 1714207.97574362
170, 89.8039921954768, 1714343.11024215
180, 1200000.0, 1714390.04694572
190, 89.8039921954766, 1714343.11024214
200, 23.1503125147703, 1714207.97574362
210, 10.8323997075252, 1714000.97899885
220, 6.55437431905477, 1713747.12816501
230, 4.61486501655445, 1713477.06832329
240, 3.6108216324324, 1713223.372641

''